# ZMUM 2: Regresja wieloraka

Prognozowanie cen domów (wiele zmiennych)

W tym notatniku wykorzystamy dane dotyczące sprzedaży domów. Przewidywanie ich ceny zostanie dokonane przy pomocy regresji wielorakiej. Pierwsze zadanie dotyczy eksploracji regresji wielorakiej poprzez opracowanie nowych cech i pomiar błędu. W drugim zadaniu zaimplementować należy algorytm spadku gradientu.

     Użyj wbudowanych funkcji, aby obliczyć współczynniki (wagi) regresji
     Biorąc pod uwagę współczynniki, predyktor i wynik napisz funkcję obliczającą resztkową sumę kwadratów (RSS)
     Spójrz na współczynniki i zinterpretuj ich znaczenie
     Oceń wiele modeli za pomocą RSS

# Potrzebne biblioteki

In [104]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


# Wczytanie danych ceny domów 

Zestaw danych pochodzi ze sprzedaży domów w King County, regionie, w którym znajduje się miasto Seattle w stanie Waszyngton.

In [105]:
dtype_dict = {'bathrooms': float, 'waterfront': int, 'sqft_above': int, 'sqft_living15': float, 'grade': int,
              'yr_renovated': int, 'price': float, 'bedrooms': float, 'zipcode': str, 'long': float,
              'sqft_lot15': float, 'sqft_living': float, 'floors': str, 'condition': int, 'lat': float, 'date': str,
              'sqft_basement': int, 'yr_built': int, 'id': str, 'sqft_lot': int, 'view': int}
domy = pd.read_csv('kc_house_data.csv', sep=',', dtype=dtype_dict)

In [106]:
domy.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


# Podziel dane na uczące i testowe.
Używamy seed = 0, aby każdy, kto korzysta z tego notebooka, uzyskał te same wyniki. W praktyce możesz ustawić losowy podział.

In [107]:
y = domy['price']
X = domy.drop(['price'], axis=1)
features = X.columns.values
features

array(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat',
       'long', 'sqft_living15', 'sqft_lot15'], dtype=object)

In [108]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Nauka modelu regresji wielorakiej

Uczymy nasz mode wykorzystując cechy 'sqft_living', 'bedrooms', 'bathrooms'.

In [109]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
pinw = np.linalg.pinv(X_train[example_features])
w = np.dot(pinw, y_train)

Po dopasowaniu modelu możemy wyodrębnić współczynniki (wagi) modelu regresji:

In [110]:
print('Otrzymane współczynniki:', w)


Otrzymane współczynniki: [   308.54966331 -41754.38204544  17354.12106635]


# Przewidywanie wartości modelu

Mając wyliczone parametry modelu napisz funkcję do przewidywania wartości dla zadanego modelu

In [111]:
def predict_output(data, w):
    res = np.dot(data, w)
    return res

In [112]:
predict_output(X_test[example_features], w)

array([ 383748.43603868, 1330310.29293091,  373818.34187199, ...,
        337749.69862805,  185213.15507236,  419805.07408762])

# Wyliczamy błąd (SSE -  sum of squared estimate of errors)

Teraz, gdy możemy wykonać przewidywanie na podstawie modelu, napiszmy funkcję obliczającą RSS modelu. Wykonaj poniższą funkcję, aby obliczy sumę kwadratów błędu estymacji (SSE) na podstawie modelu, danych i wyniku.

In [113]:
def policz_SSE(model, data, outcome):
    # Wylicz przewidywaną wartość
    y = np.dot(data, model)
    # Policz błąd bezwzglęny
    e = np.subtract(y, outcome);
    # Podnieś błędy do kwadratu i zsumuj je
    SSE = np.sum(np.power(e, 2))
    return (SSE)

Przetestuj swoją funkcję obliczając błąd SSE dla danych TEST dla przykładowego modelu:

In [114]:
przykladowe_rss = policz_SSE(w, X_test[example_features], y_test)
print(przykladowe_rss)

261738485259246.84


# Utwórz nowe cechy

Mimo iż nasz model regresji wielorakiej obejmuje wiele różnych cech (np. ilosc_sypiani, powierzchnia i ilosc_lazienek) możemy również rozważyć przekształcenie istniejących cech, np. log(powierzchnia) czy nawet mnożenie ilości sypialni i łazienek.

Użyjemy funkcji logarytmu, aby utworzyć nowe cechy, więc najpierw należy wczytać odpowiedni pakiet z biblioteki matematycznej.

In [115]:
from math import log

Następnie utwórz następujące 4 nowe cechy jako kolumny w danych TRENINGOWYCH i TESTOWYCH:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 

Jako przykład oto pierwsza:

In [116]:
X_train['bedrooms_squared'] = X_train['bedrooms'].apply(lambda x: x ** 2)
X_test['bedrooms_squared'] = X_test['bedrooms'].apply(lambda x: x ** 2)

In [117]:
# utwórz pozostałe 3 funkcje zarówno w danych TRENINGOWYCH, jak i TESTOWYCH
X_train['log_sqft_living'] = X_train['sqft_living'].apply(lambda x: log(x))
X_test['log_sqft_living'] = X_test['sqft_living'].apply(lambda x: log(x))

X_train['bed_bath_rooms'] = np.multiply(X_train['bathrooms'], X_train['bedrooms'])
X_test['bed_bath_rooms'] = np.multiply(X_test['bathrooms'], X_test['bedrooms'])

X_train['lat_plus_long'] = np.add(X_train['lat'], X_train['long'])
X_test['lat_plus_long'] = np.add(X_test['lat'], X_test['long'])

X_train

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,log_sqft_living,bed_bath_rooms,lat_plus_long
5268,5100402668,20150218T000000,3.0,1.00,1570.0,5510,1,0,0,4,...,0,98115,47.6942,-122.319,1770.0,6380.0,9.0,7.358831,3.00,-74.6248
16909,7856560480,20140808T000000,3.0,2.50,1780.0,11000,1,0,0,4,...,0,98006,47.5574,-122.149,2310.0,9700.0,9.0,7.484369,7.50,-74.5916
16123,2872900010,20150414T000000,3.0,1.50,1090.0,9862,1,0,0,3,...,0,98074,47.6256,-122.036,1710.0,9862.0,9.0,6.993933,4.50,-74.4104
12181,3216900070,20140617T000000,4.0,2.50,2210.0,7079,2,0,0,3,...,0,98031,47.4206,-122.183,1970.0,7000.0,16.0,7.700748,10.00,-74.7624
12617,0976000790,20141020T000000,3.0,2.50,1800.0,4763,2,0,0,3,...,0,98119,47.6460,-122.362,1790.0,4763.0,9.0,7.495542,7.50,-74.7160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123,2322069010,20141007T000000,5.0,5.00,3960.0,94089,2,0,0,3,...,0,98038,47.3800,-122.011,2240.0,64468.0,25.0,8.283999,25.00,-74.6310
19648,2114700368,20141118T000000,2.0,2.50,1400.0,1262,2,0,0,3,...,0,98106,47.5342,-122.349,1060.0,1524.0,4.0,7.244228,5.00,-74.8148
9845,5469501200,20140820T000000,3.0,2.25,2360.0,14950,1,0,0,4,...,0,98042,47.3856,-122.158,2720.0,14388.0,9.0,7.766417,6.75,-74.7724
10799,3751602797,20140702T000000,4.0,2.00,2370.0,76665,2,0,0,4,...,0,98001,47.2831,-122.279,2110.0,19334.0,16.0,7.770645,8.00,-74.9959


* Kwadrat sypialni zwiększa separację między nielicznymi sypialniami (np. 1) i wieloma sypialniami (np. 4), ponieważ 1 ^ 2 = 1, ale 4 ^ 2 = 16. W konsekwencji ta funkcja będzie miała wpływ głównie na domy z wieloma sypialniami.
* Sypialnia razy łazienka daje tak zwaną funkcję „interakcji”. Wynik jest wysoki, gdy  wartości *obu* cech są duże.
* Przejęcie logarytmu stóp kwadratowych powoduje zbliżenie dużych wartości i rozłożenie małych wartości. Wynika to z reguły prawoskośności posiadanych danych/posiadanego atrybutu.
* Dodawanie szerokości do długości geograficznej jest wydaje się całkowicie bezsensowne, ale i tak to zrobimy (zobaczymy później dlaczego)

**Pytanie quizu: Jakie są średnie (średnia arytmetyczna) twoich 4 nowych atrybutów w danych TEST? (w zaokrągleniu do 2 cyfr)**

In [118]:
print("bedrooms_squared: ", np.mean(X_test['bedrooms_squared']))
print("log_sqft_living: ", np.mean(X_test['log_sqft_living']))

print("bed_bath_rooms: ", np.mean(X_test['bed_bath_rooms']))
print("lat_plus_long: ", np.mean(X_test['lat_plus_long']))


bedrooms_squared:  12.210501966227158
log_sqft_living:  7.5502394677947615
bed_bath_rooms:  7.447721489706223
lat_plus_long:  -74.65426069858893


# Uczenie wielu modeli

Teraz poznamy wagi trzech (zagnieżdżonych) modeli do przewidywania cen domów. Pierwszy model będzie miał najmniej cech, drugi model powiększymy o jedną cechę, a trzeci doda jeszcze kilka:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: + bedrooms\*bathrooms
* Model 3: + log squarefeet, bedrooms squared, i (bezsensowne) latitude + longitude

In [119]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Teraz, gdy mamy już cechy, poznaj wagi trzech różnych modeli do przewidywania docelowej „ceny” za pomocą funkcji model_train i spójrz na wartość wag/współczynników:

In [120]:
# Funkcja szukająca współczynników modelu
def model_train(X_train, Y_train):
    # Wylicz współczynniki modelu wykorzystując pseudoodwrotność
    pinw = np.linalg.pinv(X_train)
    model = np.dot(pinw, Y_train)
    return (model)

In [121]:
# Wyucz 3 modele:
m1 = model_train(X_train[model_1_features], y_train)
m2 = model_train(X_train[model_2_features], y_train)
m3 = model_train(X_train[model_3_features], y_train)


In [122]:
# Wyświetl wagi współczynników modelu:
print("m1: ", m1)
print("m2: ", m2)
print("m3: ", m3)


m1:  [ 3.00159310e+02 -5.23689819e+04  4.58247068e+03  5.29238529e+05
  2.05262847e+05]
m2:  [ 2.92464746e+02 -1.21761358e+05 -1.09383881e+05  5.30036632e+05
  2.03619080e+05  3.35897011e+04]
m3:  [ 5.55032435e+02  1.34221113e+03  8.84250144e+04  3.03201909e+05
 -6.29628448e+04 -1.60202196e+04  8.18714827e+02 -6.45602857e+05
  2.40239065e+05]


**Quiz: Jaki jest znak (dodatni lub ujemny) dla współczynnika/wagi dla „łazienek” w modelu 1?**

**Quiz: Jaki jest znak (dodatni lub ujemny) dla współczynnika/wagi dla „łazienek” w modelu 2?**

Zastanów się, co to znaczy.

# Porównywanie wielu modeli

Teraz, gdy otrzymaliśmy trzy modele i wyodrębniliśmy wagi modeli, chcemy ocenić, który model jest najlepszy.

Używając wcześniej opisanych funkcji obliczy SSE dla danych uczących dla każdego z trzech modeli.

In [123]:
# Policz SSE na danych TRENINGOWYCH dla każdeg z 3 modeli i zapisz wartości:
print("m1:", policz_SSE(m1, X_train[model_1_features], y_train))
print("m2:", policz_SSE(m2, X_train[model_2_features], y_train))
print("m3:", policz_SSE(m3, X_train[model_3_features], y_train))

m1: 1082381909711003.2
m2: 1065791753438925.9
m3: 991825573399600.0


**Quiz: Który model (1, 2 lub 3) ma najniższy poziom SSE na danych TRENINGOWYCH?** Czy tego się spodziewałeś/-łaś?

Teraz obliczyć SSE na danych TEST dla każdego z trzech modeli.

In [124]:
# Teraz obliczyć SSE na danych TEST dla każdego z trzech modeli i zapisz wartości:
print("m1:", policz_SSE(m1, X_test[model_1_features], y_test))
print("m2:", policz_SSE(m2, X_test[model_2_features], y_test))
print("m3:", policz_SSE(m3, X_test[model_3_features], y_test))

m1: 233700269953884.75
m2: 229575356886062.44
m3: 221041949402646.47


**Quiz: Który model (1, 2 lub 3) ma najniższy poziom SSE na danych TESTOWYCH?** Czy tego się spodziewałeś/-łaś? Pomyśl o cechach, które zostały dodane do każdego z modeli.

# Policz pochodną

Przejdziemy teraz do obliczania pochodnej funkcji kosztu regresji. Przypomnij sobie, że funkcja kosztu jest sumą kwadratów różnic między punktami danych a przewidywanym wynikiem.

Ponieważ pochodna sumy jest sumą pochodnych, możemy obliczyć pochodną dla pojedynczego punktu danych, a następnie wyznaczyć sumę na podstawie punktów danych. Możemy zapisać kwadrat różnicy między obserwowanym a przewidywanym wynikiem dla pojedynczego punktu w następujący sposób:

(w[0]\*[CONSTANT] + w[1]\*[cecha_1] + ... + w[i] \*[cecha_i] + ... +  w[k]\*[cecha_k] - output)^2

Gdzie mamy *k* cech i stałą. Tak więc pochodną w odniesieniu do wagi w[i] według reguły łańcucha jest:

2\*(w[0]\*[CONSTANT] + w[1]\*[cecha_1] + ... + w[i] \*[cecha_i] + ... +  w[k]\*[cecha_k] - output)\* [cecha_i]

Pojęcie w nawiasach to tylko błąd (różnica między prognozowaniem a wyjściem). Możemy więc ponownie napisać to jako:

2\*error\*[cecha_i]

Oznacza to, że pochodną wagi cechy *i-tej* jest suma (ponad punkty danych) 2-krotności iloczynu błędu i samej cechy. W przypadku stałej jest to tylko dwukrotność sumy błędów!

Przypomnijmy, że dwukrotność sumy iloczynu dwóch wektorów jest tylko dwukrotnością iloczynu dwóch wektorów. Dlatego pochodna wagi dla *cechy_i* jest tylko dwukrotnością iloczynu między wartościami *cechy_i* a bieżącymi błędami.

Mając to na uwadze, należy wyznaczyć funkcję, która oblicza pochodną współczynnika na podstawie wartości cechy (we wszystkich punktach danych) i błędów (we wszystkich punktach danych).

In [125]:
def feature_derivative(errors, feature):
    # Oblicz podwojony iloczyn cech i błędów, a następnie zwróć otrzymną wartość
    derivative = 2 * np.dot(errors, feature)
    return (derivative)

# Metoda spadku gradientu / gradientu prostego (Gradient Descent)

Teraz napiszemy funkcję, która wykonuje spadek gradientu. Podstawową przesłanką jest prosta. Biorąc pod uwagę punkt początkowy, aktualizujemy bieżące wagi, przesuwając się w kierunku ujemnego gradientu. Przypomnijmy, że gradient jest kierunkiem *wzrostu*, a zatem gradient ujemny jest kierunkiem *spadku* i staramy się *zminimalizować* funkcję kosztu.

Współczynnik, z jakim poruszamy się w kierunku gradientu ujemnego, nazywa się „rozmiarem kroku” - $\alpha$. Zatrzymujemy się, gdy jesteśmy „wystarczająco blisko” rozwiązania optymalnego. Definiujemy to, wymagając, aby wielkość gradientu była mniejsza niż stała „tolerancja” - $\epsilon$.

Mając to na uwadze, wykonaj funkcję spadku gradientu poniżej, używając funkcji pochodnej. Dla każdego kroku zejścia gradientu aktualizujemy wagę każdego współczynnika przed obliczeniem naszych kryteriów zatrzymania.

In [126]:
from math import sqrt  # wywołanie funkcji na wektorze [g[0], g[1], g[2]] daje sqrt(g[0]^2 + g[1]^2 + g[2]^2)

In [127]:
import numpy as np

In [128]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)  # miej pewność, że wagi są typu numpy array
    while not converged:
        # obliczyć prognozy na podstawie feature_matrix i wag za pomocą funkcji predict_output()
        prognosis = predict_output(feature_matrix, weights)
        # wylicz błąd predykcji jako: predict_output - output
        Err_pred = np.subtract(prognosis, output)
        gradient_sum_squares = 0  # inicjuj sumę pierwiastków gradientów
        # chociaż nie osiągnęliśmy jeszcze tolerancji, zaktualizuj wagę każdej funkcji
        for i in range(len(weights)):  # zapętlenie każdej wagi
            # Przypomnij sobie, że feature_matrix [:, i] to kolumna cech powiązana z wagami[i]
            # obliczyć pochodną dla wagi[i]:
            dEdf = feature_derivative(Err_pred, feature_matrix[:, i])
            # dodaj kwadrat wartości pochodnej do sumy kwadratów gradientów (ocena zbieżności) 
            gradient_sum_squares = gradient_sum_squares + dEdf ** 2
            # odejmij wielkość kroku pomnożoną przez pochodną od bieżącej wagi
            weights[i] = weights[i] - step_size * dEdf
        # oblicz pierwiastek kwadratowy z sumy gradientów kwadratów, aby uzyskać wielkość gradientu:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True

Kilka rzeczy, na które należy zwrócić uwagę, zanim zaczniemy korzystać z metody spadku gradientu. Ponieważ gradient jest sumą wszystkich punktów danych i obejmuje iloczyn błędu i wartości cechy, sam gradient będzie bardzo duży, ponieważ cechy są duże (stopy kwadratowe), a wynik jest duży (ceny). Tak więc, chociaż można oczekiwać, że „tolerancja” będzie niewielka, mała jest jedynie zależna od wielkości cechy.

Z podobnych powodów rozmiar kroku będzie znacznie mniejszy niż można się spodziewać, ale dzieje się tak, ponieważ gradient ma tak duże wartości.

# Uruchamianie spadku gradientu jako prostej regresji

Chociaż opadanie gradientu jest zaprojektowane dla regresji wielorakiej, ponieważ stała jest teraz funkcją, możemy użyć funkcji spadku gradientu do oszacowania parametrów prostej regresji na "squarefeet". Następująca komórka ustawia funkcję parametr_macierz, wynik, wagi początkowe i rozmiar kroku dla pierwszego modelu:

In [ ]:
# Napisz funkcję get_numpy_data(train_data, simple_features, my_output) tworzącą zbiory treningowe X_train i y_train
def get_numpy_data(train_data, simple_features, my_output):
    # Napisz funkcję
    #Standaryzacja
    scaler = StandardScaler()
    Y = train_data[my_output]
    scaledY = scaler.fit_transform(Y)    
    
    X = train_data.drop([my_output], axis=1)
    scaledX = scaler.fit_transform(X)
    
    ones = np.ones((train_data.shape[0], 1))
    feature_matrix = scaledX[simple_features].to_numpy()
    simple_feature_matrix = np.hstack((ones, feature_matrix))
    output = scaledY
    return simple_feature_matrix, output

In [156]:
from sklearn.preprocessing import MinMaxScaler


# Napisz funkcję get_numpy_data(train_data, simple_features, my_output) tworzącą zbiory treningowe X_train i y_train
def get_numpy_data(train_data, simple_features, my_output):
    # Napisz funkcję
    #Standaryzacja
    scaler = MinMaxScaler()
    Y = train_data[my_output]
    scaledY = scaler.fit_transform(Y.values.reshape(-1,1)).flatten() 
    
    X = train_data.drop([my_output], axis=1)
    scaledX = scaler.fit_transform(X[simple_features])
    
    ones = np.ones((train_data.shape[0], 1))
    feature_matrix = scaledX
    simple_feature_matrix = np.hstack((ones, feature_matrix))
    output = scaledY
    return simple_feature_matrix, output
train_data = domy
# test spadku gradientów
simple_features = ['sqft_living']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-.06, 1.])
step_size = 7e-12
tolerance = 2.5e7

In [152]:
simple_feature_matrix

array([[1.        , 0.06716981],
       [1.        , 0.17207547],
       [1.        , 0.03622642],
       ...,
       [1.        , 0.05509434],
       [1.        , 0.09886792],
       [1.        , 0.05509434]])

Następnie uruchom gradient z powyższymi parametrami.

In [157]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)  # miej pewność, że wagi są typu numpy array
    while not converged:
        # obliczyć prognozy na podstawie feature_matrix i wag za pomocą funkcji predict_output()
        prognosis = predict_output(feature_matrix, weights)
        # wylicz błąd predykcji jako: predict_output - output
        Err_pred = np.subtract(prognosis, output)
        gradient_sum_squares = 0  # inicjuj sumę pierwiastków gradientów
        # chociaż nie osiągnęliśmy jeszcze tolerancji, zaktualizuj wagę każdej funkcji
        for i in range(len(weights)):  # zapętlenie każdej wagi
            # Przypomnij sobie, że feature_matrix [:, i] to kolumna cech powiązana z wagami[i]
            # obliczyć pochodną dla wagi[i]:
            dEdf = feature_derivative(Err_pred, feature_matrix[:, i])
            # dodaj kwadrat wartości pochodnej do sumy kwadratów gradientów (ocena zbieżności) 
            gradient_sum_squares = gradient_sum_squares + dEdf ** 2
            # odejmij wielkość kroku pomnożoną przez pochodną od bieżącej wagi
            weights[i] = weights[i] - step_size * dEdf
        # oblicz pierwiastek kwadratowy z sumy gradientów kwadratów, aby uzyskać wielkość gradientu:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return (weights)
a = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)

Porównaj wagi do tych uzyskanych przy pomocy pseudoodwrotności?

**Pytanie quizu: Jaka jest waga sqft_living - drugi element „simple_weights” (w zaokrągleniu do 1 miejsca po przecinku)?**

In [158]:
print(f"{a[1]:.1f}") 

1.0


Zobacz jak się zachowuje metoda gradientowa po przeskalowaniu wartości cech: